In [3]:
using CSV, DataFrames, CategoricalArrays, Plots, Statistics

seed = 42

df_train = CSV.read("data/train_tfidf.csv", DataFrame)
df_test = CSV.read("data/test_tfidf.csv", DataFrame);

In [4]:
df_train

Row,Match type,Region,day_of_week,is_weekend,month,is_public_holiday,days_to_next_course_start,tfidf_0,tfidf_1,tfidf_2,tfidf_3,tfidf_4,tfidf_5,tfidf_6,tfidf_7,tfidf_8,tfidf_9,tfidf_10,tfidf_11,tfidf_12,tfidf_13,tfidf_14,tfidf_15,tfidf_16,tfidf_17,tfidf_18,tfidf_19,tfidf_20,tfidf_21,tfidf_22,tfidf_23,tfidf_24,tfidf_25,tfidf_26,tfidf_27,tfidf_28,tfidf_29,tfidf_30,tfidf_31,tfidf_32,tfidf_33,tfidf_34,tfidf_35,tfidf_36,tfidf_37,tfidf_38,tfidf_39,tfidf_40,tfidf_41,tfidf_42,tfidf_43,tfidf_44,tfidf_45,tfidf_46,tfidf_47,tfidf_48,tfidf_49,Conv. value,Clicks
,String15,String3,String15,Bool,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,Exact match,C,Tuesday,false,12,0,62,0.809343,-0.245856,-0.249825,-0.0834383,-0.245821,0.0742169,-0.0786842,-0.0125196,-0.0304964,-0.0589995,-0.0477081,-0.016523,-0.0407291,0.0265933,-0.00202393,-0.0668369,-0.128181,-0.0594397,-0.0256092,0.0155182,0.056766,-0.000379508,0.0142239,-0.0547795,0.0145079,0.0141842,-0.0226423,-0.00421004,-0.00203714,0.0250437,0.0424606,-0.0267662,-0.023145,0.00312137,-0.0464286,0.0103656,0.021728,0.0298229,0.0687779,0.0623589,0.0192164,0.0071141,0.00666512,0.0147117,3.02401e-5,-0.00690164,0.0443226,-0.00823783,-0.0229194,-0.00725163,0.0,0
2,Phrase match,C,Monday,false,11,0,77,0.410823,-0.00307651,0.2502,-0.201192,-0.0751674,-0.0504239,0.180083,-0.0984635,0.248944,0.254649,0.172551,-0.377044,-0.14566,-0.0307238,0.0671459,0.179149,-0.0125722,0.00537451,-0.0614399,-0.0231405,-0.0465345,-0.0624447,0.0328301,0.00532505,0.00662696,-0.0205274,-0.0396169,0.0367765,0.0119843,0.116309,0.127092,0.0727087,-0.0202663,0.0146016,-0.0681915,0.0243962,-0.0255939,0.0695166,0.0195896,0.0513039,-0.099963,0.0610046,0.00706233,0.00448914,0.0229238,-0.0173032,0.0057445,0.00735909,0.00625039,0.0405718,0.0,0
3,Exact match,B,Wednesday,false,7,0,75,0.040802,0.0137657,0.0294329,0.00452686,0.016856,-0.0157859,-0.108273,0.063561,-0.101183,0.295068,-0.00185598,0.0419694,0.159813,0.0804672,0.0500514,0.0218298,0.0663045,-0.224536,-0.0806697,-0.0416093,0.0119971,0.0680496,0.025696,-0.115015,0.0960366,-0.0671458,-0.116277,-0.00136922,0.0635284,-0.102017,-0.189167,0.0597298,-0.0124011,0.0868864,-0.0458847,0.05474,0.0358097,0.0776038,0.0301542,-0.0628442,-0.0254514,0.0410914,0.022738,-0.0461513,-0.050344,0.131939,0.0222758,-0.0382957,-0.105522,0.0252524,0.0,0
4,Exact match,A,Sunday,true,7,0,85,0.809343,-0.245856,-0.249825,-0.0834383,-0.245821,0.0742169,-0.0786842,-0.0125196,-0.0304964,-0.0589995,-0.0477081,-0.016523,-0.0407291,0.0265933,-0.00202393,-0.0668369,-0.128181,-0.0594397,-0.0256092,0.0155182,0.056766,-0.000379508,0.0142239,-0.0547795,0.0145079,0.0141842,-0.0226423,-0.00421004,-0.00203714,0.0250437,0.0424606,-0.0267662,-0.023145,0.00312137,-0.0464286,0.0103656,0.021728,0.0298229,0.0687779,0.0623589,0.0192164,0.0071141,0.00666512,0.0147117,3.02401e-5,-0.00690164,0.0443226,-0.00823783,-0.0229194,-0.00725163,0.0,2
5,Broad match,USA,Monday,false,9,0,14,0.469874,-0.0685393,-0.320057,0.145933,0.358704,-0.0907951,-0.423583,-0.136045,0.0525039,-0.139637,0.0245363,-0.083907,0.0500076,-0.0142649,0.123714,0.107687,-0.203564,-0.0661185,0.0671398,0.0837512,0.0508423,0.0243354,0.000686166,-0.102627,-0.0484031,-0.0525841,0.0164538,-0.17925,-0.0729082,0.0639206,-0.0230031,-0.0515178,-0.100982,-0.109406,-0.0207891,-0.0566648,-0.132101,-0.0831701,0.0277045,0.0623318,-0.0187313,0.0521014,0.018772,0.0277825,0.00750784,0.0412629,0.0124453,-0.00538071,-0.0156156,0.00836061,0.0,0
6,Phrase match,USA,Saturday,true,2,0,9,0.185495,0.106437,0.308832,-0.0409319,0.237745,0.475534,-0.0223933,0.213249,-0.182088,-0.0556298,0.0954322,-0.148259,-0.102476,0.008904

## Predict Conversion Value

In [49]:
function get_X(df)
    features = [
        "Match type",
        "Region",
        "day_of_week",
        "is_weekend",
        "month",
        "is_public_holiday",
        "days_to_next_course_start"
    ]

    # append tfidf_0 ... tfidf_49
    append!(features, ["tfidf_$(i)" for i in 0:49])
    
    return df[!,features]
end

function get_y_conv_value(df)
    return df."Conv. value"
end

function get_y_clicks(df)
    return df.Clicks
end

function get_mse(grid, X, y)
    mse = IAI.score(grid, X, y, criterion=:mse)

    println("MSE: ", round(mse, digits=4))
    
    return mse
end

get_mse (generic function with 1 method)

In [17]:
target = "conversion"
X_train = get_X(df_train)
y_train = get_y_conv_value(df_train)

X_test = get_X(df_test)
y_test = get_y_conv_value(df_test);

In [50]:
target = "clicks"
X_train = get_X(df_train)
y_train = get_y_clicks(df_train)

X_test = get_X(df_test)
y_test = get_y_clicks(df_test);

In [52]:
grid_lr = IAI.GridSearch(
    IAI.OptimalFeatureSelectionRegressor(
        random_seed = seed
    ),
    sparsity = [5,10,15]
)

IAI.fit_cv!(grid_lr, X_train, y_train, validation_criterion=:mse, n_folds=5)
get_mse(grid_lr, X_test, y_test)

lnr = IAI.get_learner(grid_lr)
IAI.write_json("models/lr_" * target * ".json", lnr)

MSE: 0.187


26868

In [53]:
grid_oct =  IAI.GridSearch(
    IAI.OptimalTreeRegressor(
        random_seed = seed,
    ), 
    max_depth = [2,4,6,8],
    minbucket = [0.01, 0.02, 0.05],    
)

IAI.fit_cv!(grid_oct, X_train, y_train, validation_criterion=:mse, n_folds=5)
mse = get_mse(grid_oct, X_test, y_test)

lnr = IAI.get_learner(grid_oct)
IAI.write_json("models/oct_" * target * ".json", lnr)

MSE: 0.3459


10059259

In [54]:
grid_rf = IAI.GridSearch(
    IAI.RandomForestRegressor(
        random_seed=seed,
    ),
    max_depth=[2,4,6,8],
    minbucket = [0.01, 0.02, 0.05],
    num_trees = [20,25,50,100],
)

IAI.fit_cv!(grid_rf, X_train, y_train, validation_criterion=:mse, n_folds=5)
get_mse(grid_rf, X_test, y_test)

lnr = IAI.get_learner(grid_rf)
IAI.write_json("models/rf.json", lnr)

MSE: 0.2304


5566740

In [55]:
grid_xgb = IAI.GridSearch(
    IAI.XGBoostRegressor(
        random_seed=seed,
    ),
    max_depth=[2,4,6,8],
    minbucket = [0.01, 0.02, 0.05],
    num_estimators=[20,25,50,100],
)

IAI.fit_cv!(grid_xgb, X_train, y_train, validation_criterion=:mse, n_folds=5)
get_mse(grid_xgb, X_test, y_test)

lnr = IAI.get_learner(grid_xgb)
IAI.write_json("models/xgb.json", lnr)

MSE: 0.7932


831879

In [46]:
# for conversion value

get_mse(grid_lr, X_test, y_test)
get_mse(grid_oct, X_test, y_test)
get_mse(grid_rf, X_test, y_test)
get_mse(grid_xgb, X_test, y_test)

MSE: 0.0051
MSE: 0.0157
MSE: 0.0211
MSE: 0.0337


0.03372427106533249

In [56]:
# for clicks

get_mse(grid_lr, X_test, y_test)
get_mse(grid_oct, X_test, y_test)
get_mse(grid_rf, X_test, y_test)
get_mse(grid_xgb, X_test, y_test)

MSE: 0.187
MSE: 0.3459
MSE: 0.2304
MSE: 0.7932


0.7932226429863849